# *E. coli* example


In the paper where the first version of the IPA method was intially introduced ([Del Carratore et al., 2019](https://pubs.acs.org/doi/full/10.1021/acs.analchem.9b02354)), a LC/MS-based untargeted metabolomics experiment on *Escherichia coli* extract was introduced to test the method in real-life conditions.
The new version of the IPA method was applied to the datasets (positive and negative) obtained from this experiment.

## Positive dataset
The positive dataset can be found within this library:

In [1]:
import pandas as pd
dfpos = pd.read_csv('ExampleDatasets/Ecoli/Ecoli_pos.csv')
dfpos[dfpos['rel.ids']==1]

,ids,rel.ids,mzs,RTs,Int
9,2,1,116.070580,42.663120,4.527300e+09
10,54,1,117.073749,42.766938,2.749860e+08
11,73,1,231.133964,42.565073,2.357108e+08
12,221,1,70.065249,42.831656,5.957045e+07
13,484,1,118.074696,42.758915,2.231783e+07
14,2842,1,71.068563,42.741755,2.621352e+06


In order to run the method this dataset, it is necessary to load the package, the MS1 database and the adducts information.

In [2]:
from ipaPy2 import ipa
DB=pd.read_csv('DB/IPA_MS1.csv')
adducts = pd.read_csv('DB/adducts.csv')

As described in the original paper, a set of standard mixes was analysed with the same analytical setting.
Everything learned from theses samples was recorded in the .csv file shown below.

In [3]:
updates = pd.read_csv('ExampleDatasets/Beer/update_based_on_standards.csv')
updates.head()

,KEGG.id,Name,Names,Formula,monoisotopic.mass,previous.knowledge,Ref,RT,POS.adducts,main.POS.adducts,NEG.adducts,main.NEG.adducts
0,C00025,L-Glutamate,L-Glutamate;L-Glutamic acid;L-Glutaminic acid;...,C5H9N1O4,147.05316,1,Information taken by the single standard injec...,30;60,M+H;M-H2O+H;2M+H;M-NH3+H;M+Na;M+2H,M+H,M-H;M-H2O-H;M-;2M-H;M+K-2H;M-2H;3M-H,M-H
1,C00031,D-Glucose,D-Glucose;Grape sugar;Dextrose;Glucose;D-Gluco...,C6H12O6,180.06339,1,This compound has been analyses in two standar...,30;60,M+Na;M+H+Na;M+;2M+H;M+2H;M+H,M+Na,M+CH2O2-H;M-H;M+Cl;2M-H;3M-H;M-2H,M+CH2O2-H
2,C00041,L-Alanine,L-Alanine;L-2-Aminopropionic acid;L-alpha-Alanine,C3H7N1O2,89.04768,1,The standard mix containing has been analyzed ...,20;60,M+H;2M+H;M+Na;M+2Na-H;M+2H,M+H,M-H;M-;2M-H;M-2H;3M-H,M-H
3,C00042,Succinate,Succinate;Succinic acid;Butanedionic acid;Ethy...,C4H6O4,118.02661,1,Information taken by a standard mix containing...,60;100,M-H2O+H;M+H;M+Na;M-NH3+H;M+CH2+H;2M+Na;M+2H;2M+H,M-H2O+H,M-H;M-;M-H2O-H;2M-H;M-2H;3M-H,M-H
4,C00062,L-Arginine,L-Arginine;(S)-2-Amino-5-guanidinovaleric acid...,C6H14N4O2,174.11168,1,The standard mix containing has been analyzed ...,20;60,M+H;2M+H;M+Na;M+2H;2M+Na;M+H+K;M+2Na-H,M+H,M-H;2M-H;M-;M-2H;3M-H,M-H


This information can be used in the annotation process by updating the database:

In [4]:
for k in range(0,len(updates.index)):
    DB.iloc[DB['id']==updates.iloc[k,0],5] = updates.iloc[k,7]
    DB.iloc[DB['id']==updates.iloc[k,0],6] = updates.iloc[k,8]
    DB.iloc[DB['id']==updates.iloc[k,0],7] = updates.iloc[k,10]

A dataframe containing all possible biochemical connections among all metabolites present in the IPA_MS1.csv database has been pre-computed and it is available in the library. Using this instead of computing the connections will strongly speed up the pipeline.

In [5]:
Bio = pd.read_csv('DB/allBIO_reactions.csv')

Finally, we can run the whole pipeline with the simpleIPA() function.

In [6]:
annotationsPos = ipa.simpleIPA(df=dfpos,ionisation=1,DB=DB,adductsAll=adducts,ppm=5, Bio=Bio,
                            delta_add=0.1,delta_bio=0.5,burn=1000,noits=5000,ncores=1)

mapping isotope patterns ....
7.4 seconds elapsed
computing all adducts ....
186.4 seconds elapsed
annotating based on MS1 information....
114.4 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|██████████| 5000/5000 [3:35:53<00:00,  2.59s/it]  


parsing results ...
Done -  12967.1 seconds elapsed


The annotation for the same feature considered in the original paper as an example (id=2, m/z=116.070580, RT= 42.66s) and shown in [Figure 4](https://pubs.acs.org/cms/10.1021/acs.analchem.9b02354/asset/images/large/ac9b02354_0004.jpeg) is shown below.

In [7]:
annotationsPos[2]

,id,name,formula,adduct,m/z,charge,RT range,ppm,isotope pattern score,fragmentation pattern score,prior,post,post Gibbs,chi-square pval
0,C00148,L-Proline,C5H10NO2,M+H,116.070605,1,20;70,-0.214542,0.200000,None,0.194710,2.380952e-01,0.30950,5.931818e-25
2,C16435,Proline,C5H10NO2,M+H,116.070605,1,None,-0.214542,0.200000,None,0.194710,1.904762e-01,0.18875,5.931818e-25
4,NPA018555,Pleurocybellaziridin,C5H10NO2,M+H,116.070605,1,None,-0.214542,0.200000,None,0.194710,1.904762e-01,0.16850,5.931818e-25
3,C18170,3-Acetamidopropanal,C5H10NO2,M+H,116.070605,1,None,-0.214542,0.200000,None,0.194710,1.904762e-01,0.16750,5.931818e-25
1,C00763,D-Proline,C5H10NO2,M+H,116.070605,1,None,-0.214542,0.200000,None,0.194710,1.904762e-01,0.16575,5.931818e-25
5,Unknown,Unknown,None,None,None,None,None,5.000000,0.000001,None,0.026448,1.399883e-07,0.00000,5.931818e-25


The whole annotation dictonary for this dataset can be saved as a pickle file.

In [8]:
import pickle
file = open("ExampleDatasets/Ecoli/annotationsPos.pickle", "wb")
pickle.dump(annotationsPos, file)
file.close()

## Negative dataset
The negative dataset can also be found within this library:

In [9]:
dfneg = pd.read_csv('ExampleDatasets/Ecoli/Ecoli_neg.csv')
dfneg.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,191.018617,56.672309,1.790060e+09
1,14,0,192.021784,56.628663,1.244214e+08
2,30,0,111.007146,56.776775,7.138570e+07
3,75,0,193.022430,56.636817,2.568980e+07
4,177,0,405.027847,56.930517,1.097692e+07


In the same way it can be annotated with the IPA method.

In [10]:
annotationsNeg = ipa.simpleIPA(df=dfneg,ionisation=-1,DB=DB,adductsAll=adducts,ppm=10,Bio=Bio,
                            delta_add=0.1,delta_bio=0.5,burn=1000,noits=5000,ncores=1)

mapping isotope patterns ....
10.7 seconds elapsed
computing all adducts ....
1156.6 seconds elapsed
annotating based on MS1 information....
193.2 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|██████████| 5000/5000 [11:30:17<00:00,  8.28s/it]  


parsing results ...
Done -  41443.2 seconds elapsed


The whole annotation dictonary for this dataset can be saved as a pickle file.

In [11]:
file = open("ExampleDatasets/Ecoli/annotationsNeg.pickle", "wb")
pickle.dump(annotationsNeg, file)
file.close()